In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import *
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.types import IntegerType


#import nltk
#from nltk.stem import PorterStemmer
#nltk.download('wordnet')
#from nltk.stem.snowball import SnowballStemmer
#from nltk.corpus import stopwords
#from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

#import com.johnsnowlabs.nlp.base._
#import com.johnsnowlabs.nlp.annotator._

import sparknlp
from sparknlp.pretrained import PretrainedPipeline

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import *
from pyspark.sql.functions import rand,when

In [2]:
spark.version

Out[4]: '2.4.4'

In [3]:
spark=SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
df1=spark.read.table('table1')
df3=spark.read.table('table3')
#df1.count()
#df3.count()
dfinal= df1.union(df3)
#dfinal.count()
#dfinal.show(5)

In [5]:
dpandas = dfinal.select('*').toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [6]:
dpandas['content'] = dpandas['content'].str.replace('\W', ' ')
dpandas['title'] = dpandas['title'].str.replace('\W', ' ')
dpandas['content'] = dpandas['content'].str.replace(' +', ' ')

In [7]:
dfSpark = spark.createDataFrame(dpandas)
#dfSpark.show(5)

In [8]:
tokenization=Tokenizer(inputCol='content',outputCol='tokens')
tokenized_df=tokenization.transform(dfSpark)
tokenized_df.show(10)

+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+--------------------+
_c0| id| title| publication| author| date| year|month| url| content| tokens|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+--------------------+
 0|17283|House Republicans...|New York Times| Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON Congre...|[washington, cong...|
 1|17284|Rift Between Offi...|New York Times|Benjamin Mueller ...|2017-06-19|2017.0| 6.0|null|After the bullet ...|[after, the, bull...|
 2|17285|Tyrus Wong Bamb...|New York Times| Margalit Fox|2017-01-06|2017.0| 1.0|null|When Walt Disney ...|[when, walt, disn...|
 3|17286|Among Deaths in 2...|New York Times| William McDonald|2017-04-10|2017.0| 4.0|null|Death may be the ...|[death, may, be, ...|
 4|17287|Kim Jong un Says ...|New York Times| Choe Sang-Hun|2017-01-02|2017.0| 1.0|null|SEOUL South Korea...|[seoul, south, ko...|
 5|17288|Sick With a Cold ...|New York Times| Sewell Chan|2017-01-02|2017.0| 1.0|null|LONDON Queen Eliz...|[london, queen, e...|
 6|17289|Taiwan s Presiden...|New York Times| Javier C. Hernández|2017-01-02|2017.0| 1.0|null|BEIJING President...|[beijing, preside...|
 7|17290|After The Bigges...|New York Times| Gina Kolata|2017-02-08|2017.0| 2.0|null|Danny Cahill stoo...|[danny, cahill, s...|
 8|17291|First a Mixtape ...|New York Times| Katherine Rosman|2016-12-31|2016.0| 12.0|null|Just how is Hilla...|[just, how, is, h...|
 9|17292|Calling on Angels...|New York Times| Andy Newman|2016-12-31|2016.0| 12.0|null|Angels are everyw...|[angels, are, eve...|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+--------------------+
only showing top 10 rows

In [9]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [10]:
dfSpark=stopword_removal.transform(tokenized_df)

In [11]:
dfSpark.select(['id','tokens','refined_tokens']).show(10)

+-----+--------------------+--------------------+
 id| tokens| refined_tokens|
+-----+--------------------+--------------------+
17283|[washington, cong...|[washington, cong...|
17284|[after, the, bull...|[bullet, shells, ...|
17285|[when, walt, disn...|[walt, disney, ba...|
17286|[death, may, be, ...|[death, may, grea...|
17287|[seoul, south, ko...|[seoul, south, ko...|
17288|[london, queen, e...|[london, queen, e...|
17289|[beijing, preside...|[beijing, preside...|
17290|[danny, cahill, s...|[danny, cahill, s...|
17291|[just, how, is, h...|[hillary, kerr, f...|
17292|[angels, are, eve...|[angels, everywhe...|
+-----+--------------------+--------------------+
only showing top 10 rows

In [12]:
dfSpark.filter(((dfSpark.publication =='New York Times') | (dfSpark.publication =='Vox')))

Out[14]: DataFrame[_c0: string, id: string, title: string, publication: string, author: string, date: string, year: string, month: string, url: string, content: string, tokens: array<string>, refined_tokens: array<string>]

In [13]:
display(dfSpark.groupBy('publication').count())
#display(dfSpark.groupBy('year').count())


publication,count
Business Insider,6757
Washington Post,11114
Reuters,10710
CNN,11488
New York Times,7803
Guardian,3808
Breitbart,23781
NPR,11992
Atlantic,171
Vox,4947


In [14]:
sparknlp.start()
pipeline = PretrainedPipeline('analyze_sentiment', 'en')

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [15]:
dfSpark = dfSpark.withColumn('Sentimiento', when(rand() > 0.5, 1).otherwise(0))

In [16]:
dfSpark = dfSpark.withColumn("label", dfSpark.Sentimiento.cast('float')).drop('Sentimiento')

In [17]:
dfSpark.orderBy(rand()).show(10)

+------+------+--------------------+----------------+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+-----+
 _c0| id| title| publication| author| date| year|month| url| content| tokens| refined_tokens|label|
+------+------+--------------------+----------------+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+-----+
145486|217370|U S economy adde...| Washington Post| Ana Swanson|2016-12-02|2016.0| 12.0|https://web.archi...| The U S economy ...|[, the, u, s, eco...|[, u, economy, ad...| 1.0|
116358|171816|Research On Tulsa...| NPR| Claudio Sanchez|2016-08-22|2016.0| 8.0|http://www.npr.or...|In 1998 Oklahoma ...|[in, 1998, oklaho...|[1998, oklahoma, ...| 0.0|
 48557| 67927|Russia s ideal ...|Business Insider| Natasha Bertrand|2016-01-29|2016.0| 1.0| null| Syria s main opp...|[, syria, s, main...|[, syria, main, o...| 0.0|
110004|164087|Brad Paisley On ...| NPR| Vince Pearson|2017-04-24|2017.0| 4.0|http://www.npr.or...|Brad Paisley is a...|[brad, paisley, i...|[brad, paisley, c...| 0.0|
137987|207829|The best veggie b...| Washington Post| Maura Judkis|2017-05-25|2017.0| 5.0|https://web.archi...| Beef not your th...|[, beef, not, you...|[, beef, thing, f...| 1.0|
 31133| 49898|Spike Lee Electi...| Breitbart| Kipp Jones|2016-02-16|2016.0| 2.0| null|Director Spike Le...|[director, spike,...|[director, spike,...| 0.0|
126803|189535|Exclusive North ...| Reuters|David Brunnstrom ...|2016-06-14|2016.0| 6.0|http://www.reuter...| North Korea may ...|[, north, korea, ...|[, north, korea, ...| 1.0|
 49579| 69098|Warriors GM had a...|Business Insider| Scott Davis|2016-04-12|2016.0| 4.0| null| On Sunday night ...|[, on, sunday, ni...|[, sunday, night,...| 1.0|
 8403| 27139|Inauguration Cove...| Breitbart|Breitbart News, A...|2017-01-21|2017.0| 1.0| null|Welcome to Breitb...|[welcome, to, bre...|[welcome, breitba...| 0.0|
 12886| 31622|Google Launches A...| Breitbart| Lucas Nolan|2017-02-23|2017.0| 2.0| null|Google has launch...|[google, has, lau...|[google, launched...| 1.0|
+------+------+--------------------+----------------+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 10 rows

In [18]:
length = dfSpark.count()
lista_sentimientos = []
#Como parametro de range solo seria necesario cambiarlo por la variable LENGTH para que evalue todo el dataset
for i in range(3):
  columna_content = dfSpark.select('content').collect()[i].__getitem__("content")
  pred = pipeline.annotate(columna_content)
  if pred['sentiment'][0] == 'positive':
    lista_sentimientos.append('1')
  elif pred['sentiment'][0] == 'negative':
    lista_sentimientos.append('0')
  #print(pred['sentiment'])
#print(lista_sentimientos)
spark.createDataFrame(lista_sentimientos, StringType()).show()
#print(dfSpark.show())

+-----+
value|
+-----+
 1|
 0|
 1|
+-----+

In [19]:
len_udf = udf(lambda s: len(s), IntegerType())
dfSpark = dfSpark.withColumn("token_count", len_udf(col('refined_tokens')))
dfSpark.orderBy(rand()).show(10)

+------+------+--------------------+----------------+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+-----+-----------+
 _c0| id| title| publication| author| date| year|month| url| content| tokens| refined_tokens|label|token_count|
+------+------+--------------------+----------------+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+-----+-----------+
 5024| 22864|The Electoral Col...| New York Times| Emmarie Huetteman|2016-12-20|2016.0| 12.0| null|Follow the latest...|[follow, the, lat...|[follow, latest, ...| 0.0| 366|
128535|192607|U S telecom firm...| Reuters|Liana B. Baker an...|2016-11-07|2016.0| 11.0|http://www.reuter...|The merger if com...|[the, merger, if,...|[merger, complete...| 0.0| 306|
 28061| 46821|Jeb Bush Runs To ...| Breitbart| Alex Swoyer|2015-12-31|2015.0| 12.0| null|As GOP presidenti...|[as, gop, preside...|[gop, presidentia...| 0.0| 130|
 49460| 68959|Climate change is...|Business Insider| Mark Abadi|2016-04-03|2016.0| 4.0| null| The global winem...|[, the, global, w...|[, global, winema...| 1.0| 99|
113416|168175|Marines Gear Up F...| NPR| Tom Bowman|2016-03-28|2016.0| 3.0|http://www.npr.or...|The Marines will ...|[the, marines, wi...|[marines, begin, ...| 1.0| 374|
110861|165197|Dozens Of Deaths ...| NPR| Colin Dwyer|2017-06-01|2017.0| 6.0|http://www.npr.or...|Updated 5 15 p m ...|[updated, 5, 15, ...|[updated, 5, 15, ...| 0.0| 258|
 14810| 33546|Nancy Pelosi Fear...| Breitbart| Breitbart News|2017-02-15|2017.0| 2.0| null|Mike Lillis repor...|[mike, lillis, re...|[mike, lillis, re...| 0.0| 106|
119877|176725|Mary Tyler Moore ...| Reuters| Bill Trott|2017-01-26|2017.0| 1.0|http://www.reuter...|Moore who won sev...|[moore, who, won,...|[moore, won, seve...| 0.0| 647|
145747|217706|Clinton blames Pu...| Washington Post| Abby Phillip|2016-12-16|2016.0| 12.0|https://web.archi...| At a thank you e...|[, at, a, thank, ...|[, thank, event, ...| 0.0| 323|
 49133| 68584|Citigroup has set...|Business Insider| Matt Turner|2016-03-10|2016.0| 3.0| null| Citigroup has se...|[, citigroup, has...|[, citigroup, set...| 1.0| 52|
+------+------+--------------------+----------------+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+-----+-----------+
only showing top 10 rows

In [20]:
from pyspark.ml.feature import CountVectorizer

In [21]:
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')
dfSpark_V=count_vec.fit(dfSpark).transform(dfSpark)
dfSpark_V.select(['refined_tokens','token_count','features','Label']).show(10)

+--------------------+-----------+--------------------+-----+
 refined_tokens|token_count| features|Label|
+--------------------+-----------+--------------------+-----+
[washington, cong...| 498|(249350,[0,1,2,3,...| 1.0|
[bullet, shells, ...| 2632|(249350,[0,2,3,4,...| 0.0|
[walt, disney, ba...| 1332|(249350,[0,2,3,4,...| 0.0|
[death, may, grea...| 1206|(249350,[0,2,3,4,...| 1.0|
[seoul, south, ko...| 429|(249350,[0,1,4,5,...| 0.0|
[london, queen, e...| 90|(249350,[0,4,5,9,...| 0.0|
[beijing, preside...| 347|(249350,[0,1,2,3,...| 1.0|
[danny, cahill, s...| 1677|(249350,[0,2,3,4,...| 1.0|
[hillary, kerr, f...| 893|(249350,[0,2,3,4,...| 0.0|
[angels, everywhe...| 437|(249350,[0,2,3,4,...| 0.0|
+--------------------+-----------+--------------------+-----+
only showing top 10 rows

In [22]:
model_df=dfSpark_V.select(['features','token_count','Label'])

In [23]:
from pyspark.ml.feature import VectorAssembler

In [24]:
df_assembler = VectorAssembler(inputCols=['features','token_count'],outputCol='features_vec')
model_df = df_assembler.transform(model_df)

In [25]:

model_df.printSchema()

root
-- features: vector (nullable = true)
-- token_count: integer (nullable = true)
-- Label: float (nullable = false)
-- features_vec: vector (nullable = true)

In [26]:
from pyspark.ml.classification import LogisticRegression

In [27]:
training_df,test_df=model_df.randomSplit([0.25,0.75])

In [28]:
training_df.groupBy('Label').count().show()

In [29]:
test_df.groupBy('Label').count().show()

In [30]:
log_reg=LogisticRegression(featuresCol='features_vec',labelCol='Label').fit(training_df)
results=log_reg.evaluate(test_df).predictions
results.show()

In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [32]:
true_postives = results[(results.Label == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.Label == 0) & (results.prediction == 0)].count()
false_positives = results[(results.Label == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.Label == 1) & (results.prediction == 0)].count()

In [33]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

In [34]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

In [35]:
accuracy=float((true_postives+true_negatives) /(results.count()))
print(accuracy)